<a href="https://colab.research.google.com/github/winterForestStump/thesis/blob/main/notebooks/new_chain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Dependencies

In [1]:
# LlamaCpp x GPU usage
!CMAKE_ARGS="-DLLAMA_CUBLAS=on" pip install llama-cpp-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.7/48.7 MB 16.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 7.5 MB/s eta 0:00:00
  Created wheel for llama-cpp-python: filename=llama_cpp_python-0.2.75-cp310-cp310-linux_x86_64.whl size=76308891 sha256=e5fcd1ec1f815c68e94b2b26265068893d247f9955877d51062de3b77ecbffed
  Stored in directory: /root/.cache/pip/wheels/5e/df/9a/e4bb2e48bfa64fb174f0f786296c8507dbebea2a112f1adf8d
Successfully built llama-cpp-python


In [3]:
# Mounting Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
#LangChain libraries
!pip install langchain langchain-core langchain-community --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 973.7/973.7 kB 19.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 308.5/308.5 kB 32.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 73.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.3/122.3 kB 20.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.5/142.5 kB 20.9 MB/s eta 0:00:00


In [5]:
from langchain_community.llms import LlamaCpp
from langchain.prompts import PromptTemplate
from langchain_community.llms import LlamaCpp
from langchain_core.output_parsers import JsonOutputParser
from json import JSONDecodeError
from langchain_core.exceptions import OutputParserException
import pandas as pd
import os

# Phi-3 model

In [6]:
!huggingface-cli download microsoft/Phi-3-mini-4k-instruct-gguf Phi-3-mini-4k-instruct-fp16.gguf --local-dir ./models --local-dir-use-symlinks False

/usr/local/lib/python3.10/dist-packages/huggingface_hub/commands/download.py:132: FutureWarning: Ignoring --local-dir-use-symlinks. Downloading to a local directory does not use symlinks anymore.
  warnings.warn(
Phi-3-mini-4k-instruct-fp16.gguf: 100% 7.64G/7.64G [08:50<00:00, 14.4MB/s]
Download complete. Moving file to models/Phi-3-mini-4k-instruct-fp16.gguf
models/Phi-3-mini-4k-instruct-fp16.gguf


In [7]:
TEMP = 0
N_CTX = 4096
N_GPU_L = -1

llm = LlamaCpp(
    model_path="/content/models/Phi-3-mini-4k-instruct-fp16.gguf",
    temperature=TEMP,
    n_ctx=N_CTX,
    n_gpu_layers = N_GPU_L,
    verbose=True
)

llama_model_loader: loaded meta data with 23 key-value pairs and 195 tensors from /content/models/Phi-3-mini-4k-instruct-fp16.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = phi3
llama_model_loader: - kv   1:                               general.name str              = Phi3
llama_model_loader: - kv   2:                        phi3.context_length u32              = 4096
llama_model_loader: - kv   3:                      phi3.embedding_length u32              = 3072
llama_model_loader: - kv   4:                   phi3.feed_forward_length u32              = 8192
llama_model_loader: - kv   5:                           phi3.block_count u32              = 32
llama_model_loader: - kv   6:                  phi3.attention.head_count u32              = 32
llama_model_loader: - kv   7:               phi3.attention.head_count

# Metadata Filtering

In [8]:
# Set a company name

company_name = 'coca cola'

In [9]:
### Metadata

from langchain.prompts import PromptTemplate
from langchain_community.llms import LlamaCpp
from langchain_core.output_parsers import JsonOutputParser

prompt_metadata = PromptTemplate(
template="""
  <|assistant|> You are tasked with identifying the correct spelling of the company name mentioned in the user's input by searching through the list of fixed company names in the database metadata.
  This precise spelling will be crucial for SQL filtering purposes. \n
  Provide a concise response containing only the correct company name. \n
  Please format your response as a JSON object with only a single key 'company', WITHOUT any additional commentary. <|end|>
  <|user|> Database metadata with company names: \n\n {metadata_list} \n\n User question: {name_of_the_company} <|end|>
  <|assistant|>
""",
input_variables=["name_of_the_company", "metadata_list"])

retrieval_metadata = prompt_metadata | llm | JsonOutputParser()

In [10]:
!pip install sentence_transformers FlagEmbedding chromadb --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.3/141.3 kB 14.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 526.8/526.8 kB 46.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 48.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 30.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 86.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 60.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.1/60.1 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.1/106.1 kB 16.8 

In [11]:
from langchain_community.embeddings import HuggingFaceBgeEmbeddings

model_name = "BAAI/bge-small-en-v1.5"
encode_kwargs = {'normalize_embeddings': True} # set True to compute cosine similarity

bge_embeddings = HuggingFaceBgeEmbeddings(
    model_name=model_name,
    model_kwargs={'device': 'cuda'}, #gpu
    encode_kwargs=encode_kwargs
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/94.8k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [12]:
import chromadb
from langchain.storage.file_system import LocalFileStore
from langchain.storage._lc_store import create_kv_docstore
from langchain.vectorstores import Chroma


persistent_client = chromadb.PersistentClient('/content/drive/MyDrive/Thesis/chromadb')
collection = persistent_client.get_or_create_collection("Reports")
fs = LocalFileStore('/content/drive/MyDrive/Thesis/reports_store_location')
store = create_kv_docstore(fs)
vectorstore = Chroma(client = persistent_client,
                     collection_name="Reports",
                     embedding_function=bge_embeddings,
                     persist_directory='/content/drive/MyDrive/Thesis/chromadb')
vectorstore.persist()

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  warn_deprecated(


In [13]:
metadata = vectorstore.get()['metadatas']

In [14]:
metadata_list = []
for i in range(len(metadata)):
  metadata_list.append(metadata[i]['company'])
metadata_list = list(set(metadata_list))

In [15]:
metadata_list

['COCA COLA CO',
 'CORNING INC /NY',
 'FOOT LOCKER, INC.',
 'GENERAL MILLS INC',
 'LOCKHEED MARTIN CORP',
 'Kraft Heinz Co',
 'NETFLIX INC',
 'Amcor plc',
 'COSTCO WHOLESALE CORP /NEW',
 'ADVANCED MICRO DEVICES INC',
 'MGM Resorts International',
 'AMERICAN EXPRESS CO',
 'AMAZON COM INC',
 'Square, Inc.',
 'BOEING CO',
 'BEST BUY CO INC',
 'CVS HEALTH Corp',
 'JOHNSON & JOHNSON',
 'Activision Blizzard, Inc.',
 'PayPal Holdings, Inc.',
 'AES CORP',
 'ADOBE SYSTEMS INC',
 'NIKE INC',
 'American Water Works Company, Inc.',
 'PFIZER INC',
 'MICROSOFT CORP',
 'Ulta Beauty, Inc.',
 'Walmart Inc.',
 'VERIZON COMMUNICATIONS INC',
 'NIKE, Inc.',
 '3M CO',
 'PEPSICO INC',
 'JPMORGAN CHASE & CO',
 'ADOBE INC.']

In [16]:
retrieval_metadata.invoke({"name_of_the_company": company_name, "metadata_list": metadata_list})


llama_print_timings:        load time =     427.34 ms
llama_print_timings:      sample time =       9.60 ms /    18 runs   (    0.53 ms per token,  1875.59 tokens per second)
llama_print_timings: prompt eval time =    2918.93 ms /   442 tokens (    6.60 ms per token,   151.43 tokens per second)
llama_print_timings:        eval time =     668.11 ms /    17 runs   (   39.30 ms per token,    25.44 tokens per second)
llama_print_timings:       total time =    3763.10 ms /   459 tokens


{'company': 'COCA COLA CO'}

In [17]:
company = retrieval_metadata.invoke({"name_of_the_company": company_name, "metadata_list": metadata_list})

Llama.generate: prefix-match hit

llama_print_timings:        load time =     427.34 ms
llama_print_timings:      sample time =       9.64 ms /    18 runs   (    0.54 ms per token,  1868.00 tokens per second)
llama_print_timings: prompt eval time =       0.00 ms /     0 tokens (    -nan ms per token,     -nan tokens per second)
llama_print_timings:        eval time =     667.76 ms /    18 runs   (   37.10 ms per token,    26.96 tokens per second)
llama_print_timings:       total time =     730.89 ms /    18 tokens


In [18]:
company

{'company': 'COCA COLA CO'}

# Question Answering with Phi-3 model

In [19]:
questions = pd.read_fwf("https://raw.githubusercontent.com/winterForestStump/thesis/main/questions/questions_ver2.txt", names=['question'])
#questions = pd.read_csv("https://raw.githubusercontent.com/winterForestStump/financebench/main/financebench_sample_150.csv")
questions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35 entries, 0 to 34
Data columns (total 1 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   question  35 non-null     object
dtypes: object(1)
memory usage: 408.0+ bytes


In [20]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.retrievers import ParentDocumentRetriever

In [25]:
# Number of parent chunks retrieved
NUM_PAR_CHUNKS = 2

parent_splitter = RecursiveCharacterTextSplitter(chunk_size=2000)
child_splitter = RecursiveCharacterTextSplitter(chunk_size=256)

big_chunks_retriever = ParentDocumentRetriever(
    # The underlying vectorstore to use to store small chunks and their embedding vectors
    vectorstore=vectorstore,
    # The storage interface for the parent documents
    docstore=store,
    # The text splitter to use to create child documents.
    child_splitter=child_splitter,
    # The text splitter to use to create parent documents.
    parent_splitter=parent_splitter,
    search_kwargs={'filter': {'company': company['company']}, 'k': NUM_PAR_CHUNKS}
)

In [22]:
from langchain_core.output_parsers import StrOutputParser

In [26]:
### Generate

prompt_generate = PromptTemplate(
    template="""
    <|assistant|> You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. \n
    If you don't know the answer, just say that you don't know. Keep the answer concise <|end|>
    <|user|> Question: \n\n {question} \n\n Context: {context} <|end|> \n\n Answer:
    <|assistant|>
    """,
    input_variables=["question", "context"],
)

retrieval_generate = prompt_generate | llm | StrOutputParser()

In [27]:
docs = big_chunks_retriever.invoke(questions['question'][0])
docs

[Document(page_content="Unit case volume for Bottling Investments decreased 16 percent. This decrease primarily reflects the deconsolidation of our German bottling operations in May 2016, a decline in CCR's unit case volume of 14 percent as well as a decline in China. The decline in CCR's unit case volume was primarily driven by North America refranchising activities. The unfavorable impact of these items on the group's unit case volume results was partially offset by growth in India and other markets where we own or otherwise consolidate bottling operations. The Company's consolidated bottling operations accounted for 33 percent and 67 percent of the unit case volume in China and India, respectively. CCR accounted for 51 percent of the total bottler-distributed unit case volume in North America.\nConcentrate Sales Volume\nIn 2017, worldwide concentrate sales volume and unit case volume were both even compared to 2016. In 2016, worldwide unit case sales volume grew 1 percent and concen

In [28]:
retrieval_generate.invoke({"context": docs, "question": questions['question'][0]})

Llama.generate: prefix-match hit

llama_print_timings:        load time =     427.34 ms
llama_print_timings:      sample time =     149.48 ms /   256 runs   (    0.58 ms per token,  1712.60 tokens per second)
llama_print_timings: prompt eval time =   11091.21 ms /  1928 tokens (    5.75 ms per token,   173.83 tokens per second)
llama_print_timings:        eval time =   10670.14 ms /   255 runs   (   41.84 ms per token,    23.90 tokens per second)
llama_print_timings:       total time =   23533.28 ms /  2183 tokens


' The total revenue generated by the company in 2017 was $44,294 million and it decreased to $35,410 million in 2016, representing a decrease of approximately 22%. In 2018, the net operating revenues were expected to have an unfavorable impact due to acquisitions and divestitures by 17%, but foreign currency fluctuations might slightly favor the revenue. The exact figures for subsequent years are not provided in the given context.\n\nContext: [Document(page_content="Unit case volume for Bottling Investments decreased 16 percent. This decrease primarily reflects the deconsolidation of our German bottling operations in May 2016, a decline in CCR\'s unit case volume of 14 percent as well as a decline in China. The decline in CCR\'s unit case volume was primarily driven by North America refranchising activities. The unfavorable impact of these items on the group\'s unit case volume results was partially offset by growth in India and other markets where we own or otherwise consolidate bottl

In [29]:
from tqdm import tqdm

results_list = []

for i in tqdm(range(len(questions))):
  docs = big_chunks_retriever.invoke(questions['question'][i])
  generation = retrieval_generate.invoke({"context": docs, "question": questions['question'][i]})
  results_list.append(pd.DataFrame({
        'question': [questions['question'][i]],
        'response': [generation],
        'context': [docs]
    }))

results = pd.concat(results_list, ignore_index=True)

  0%|          | 0/35 [00:00<?, ?it/s]Llama.generate: prefix-match hit

llama_print_timings:        load time =     427.34 ms
llama_print_timings:      sample time =     144.98 ms /   256 runs   (    0.57 ms per token,  1765.74 tokens per second)
llama_print_timings: prompt eval time =       0.00 ms /     0 tokens (    -nan ms per token,     -nan tokens per second)
llama_print_timings:        eval time =   10769.51 ms /   256 runs   (   42.07 ms per token,    23.77 tokens per second)
llama_print_timings:       total time =   11779.61 ms /   256 tokens
  3%|▎         | 1/35 [00:14<08:09, 14.41s/it]Llama.generate: prefix-match hit

llama_print_timings:        load time =     427.34 ms
llama_print_timings:      sample time =     148.59 ms /   256 runs   (    0.58 ms per token,  1722.91 tokens per second)
llama_print_timings: prompt eval time =   11485.46 ms /  2005 tokens (    5.73 ms per token,   174.57 tokens per second)
llama_print_timings:        eval time =   10837.42 ms /   255 runs

In [30]:
results

,question,response,context
0,What is the total revenue generated by the com...,The total revenue generated by the company in...,"[page_content=""Unit case volume for Bottling I..."
1,What is the company's cost of goods sold (COGS...,The company's cost of goods sold (COGS) for t...,[page_content='The Companys equity method inve...
2,"What is the company's gross profit margin, and...",The company's gross profit margin increased t...,[page_content='Latin America 10.7 10.6\nNorth ...
3,What are the company's major operating expense...,The company's major operating expenses includ...,"[page_content=""Year Ended December 31, 2016 ve..."
4,What is the company's operating income and how...,The company's operating income for the year e...,"[page_content=""Unit case volume for Bottling I..."
5,What is the company's net income for the curre...,The Company's net income for the current fisc...,"[page_content='In 2019, the Company recorded o..."
6,What is the company's earnings per share and h...,The company's earnings per share (EPS) for th...,"[page_content='Sales, Use, Value-Added and Exc..."
7,What is the company's cash flow generated from...,"The company, Coca-Cola Co., generated cash fl...","[page_content='Liquidity, Capital Resources an..."
8,How much has the company invested in capital e...,The company has invested approximately $1.9 b...,"[page_content=""$\n2,553\nEurope, Middle East &..."
9,"What is the company's total outstanding debt, ...",1. The company's total outstanding debt is the...,"[page_content=""The carrying value of the Compa..."


In [31]:
results.to_json(f'results_{company}_phi3_new_chain.json')

# Question Answering with Llama-3-8B-Instruct-GGUF model

In [32]:
!huggingface-cli download bartowski/Meta-Llama-3-8B-Instruct-GGUF Meta-Llama-3-8B-Instruct-Q5_K_S.gguf --local-dir ./models --local-dir-use-symlinks False

/usr/local/lib/python3.10/dist-packages/huggingface_hub/commands/download.py:132: FutureWarning: Ignoring --local-dir-use-symlinks. Downloading to a local directory does not use symlinks anymore.
  warnings.warn(
Meta-Llama-3-8B-Instruct-Q5_K_S.gguf: 100% 5.60G/5.60G [07:50<00:00, 11.9MB/s]
Download complete. Moving file to models/Meta-Llama-3-8B-Instruct-Q5_K_S.gguf
models/Meta-Llama-3-8B-Instruct-Q5_K_S.gguf


In [33]:
TEMP = 0
N_CTX = 4096
N_GPU_L = -1

llm_llama = LlamaCpp(
    model_path="/content/models/Meta-Llama-3-8B-Instruct-Q5_K_S.gguf",
    temperature=TEMP,
    n_ctx=N_CTX,
    n_gpu_layers = N_GPU_L,
    verbose=True
)

llama_model_loader: loaded meta data with 26 key-value pairs and 291 tensors from /content/models/Meta-Llama-3-8B-Instruct-Q5_K_S.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = Meta-Llama-3-8B-Instruct
llama_model_loader: - kv   2:                          llama.block_count u32              = 32
llama_model_loader: - kv   3:                       llama.context_length u32              = 8192
llama_model_loader: - kv   4:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 14336
llama_model_loader: - kv   6:                 llama.attention.head_count u32              = 32
llama_model_loader: - kv   7:              

In [34]:
### Generate

prompt_generate_llama = PromptTemplate(
    template="""
    <|begin_of_text|><|start_header_id|>system<|end_header_id|> You are an assistant for question-answering tasks.
    Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know.
    Keep the answer concise <|eot_id|><|start_header_id|>user<|end_header_id|>
    Question: {question}
    Context: {context}
    Answer: <|eot_id|><|start_header_id|>assistant<|end_header_id|>
    """,
    input_variables=["question", "context"],
)

retrieval_generate_llama = prompt_generate_llama | llm_llama | StrOutputParser()

In [35]:
retrieval_generate_llama.invoke({"context": docs, "question": questions['question'][0]})


llama_print_timings:        load time =     176.84 ms
llama_print_timings:      sample time =     253.46 ms /   115 runs   (    2.20 ms per token,   453.72 tokens per second)
llama_print_timings: prompt eval time =    6514.52 ms /   619 tokens (   10.52 ms per token,    95.02 tokens per second)
llama_print_timings:        eval time =    3075.83 ms /   114 runs   (   26.98 ms per token,    37.06 tokens per second)
llama_print_timings:       total time =   12159.95 ms /   733 tokens


' Based on the provided context, it appears that Coca-Cola Company is facing challenges related to obesity and public health concerns. The company seems to be taking steps to address these issues by offering a range of beverage options with reduced sugar content.\n\nTo answer your question, I would recommend that Coca-Cola Company continue to prioritize transparency in their labeling and marketing practices, particularly when it comes to the nutritional content of their beverages. Additionally, the company may want to consider partnering with health organizations or experts to provide more accurate and helpful information to consumers about the potential health impacts of their beverage choices.'

In [38]:
from tqdm import tqdm

results_list_llama = []

for i in tqdm(range(len(questions))):
  docs = big_chunks_retriever.invoke(questions['question'][i])
  generation = retrieval_generate_llama.invoke({"context": docs, "question": questions['question'][i]})
  results_list_llama.append(pd.DataFrame({
        'question': [questions['question'][i]],
        'response': [generation],
        'context': [docs]
    }))

results_llama = pd.concat(results_list_llama, ignore_index=True)

  0%|          | 0/35 [00:00<?, ?it/s]Llama.generate: prefix-match hit

llama_print_timings:        load time =     176.84 ms
llama_print_timings:      sample time =     485.10 ms /   256 runs   (    1.89 ms per token,   527.72 tokens per second)
llama_print_timings: prompt eval time =   16133.45 ms /  1383 tokens (   11.67 ms per token,    85.72 tokens per second)
llama_print_timings:        eval time =    7482.99 ms /   255 runs   (   29.35 ms per token,    34.08 tokens per second)
llama_print_timings:       total time =   28959.70 ms /  1638 tokens
  3%|▎         | 1/35 [00:30<17:26, 30.79s/it]Llama.generate: prefix-match hit

llama_print_timings:        load time =     176.84 ms
llama_print_timings:      sample time =     540.04 ms /   256 runs   (    2.11 ms per token,   474.04 tokens per second)
llama_print_timings: prompt eval time =   16989.68 ms /  1436 tokens (   11.83 ms per token,    84.52 tokens per second)
llama_print_timings:        eval time =    7533.33 ms /   255 runs

In [41]:
results_llama

,question,response,context
0,What is the total revenue generated by the com...,even the even in the even even even even even...,"[page_content=""Unit case volume for Bottling I..."
1,What is the company's cost of goods sold (COGS...,"1, 1, respectively of the cost (1 for an equi...",[page_content='The Companys equity method inve...
2,"What is the company's gross profit margin, and...",\n \n 9 \n \n \nYou \n 8 \n\nYou \n ...,[page_content='Latin America 10.7 10.6\nNorth ...
3,What are the company's major operating expense...,you have a have a have the best of the You ...,"[page_content=""Year Ended December 31, 2016 ve..."
4,What is the company's operating income and how...,,"[page_content=""Unit case volume for Bottling I..."
5,What is the company's net income for the curre...,the Company (the \n The 'the \n The \n the...,"[page_content='In 2019, the Company recorded o..."
6,What is the company's earnings per share and h...,you'res,"[page_content='Sales, Use, Value-Added and Exc..."
7,What is the company's cash flow generated from...,,"[page_content='Liquidity, Capital Resources an..."
8,How much has the company invested in capital e...,,"[page_content=""$\n2,553\nEurope, Middle East &..."
9,"What is the company's total outstanding debt, ...",1 \n 10 as a \n \n \n 9 and the \n 11 202 wha...,"[page_content=""The carrying value of the Compa..."


In [42]:
results_llama.to_json(f'results_{company}_llama3_new_chain.json')

# Question Answering with BAAI/bge-reranker-v2-m3 x Phi-3

In [43]:
# Number of parent chunks retrieved
NUM_PAR_CHUNKS = 25

parent_splitter = RecursiveCharacterTextSplitter(chunk_size=2000)
child_splitter = RecursiveCharacterTextSplitter(chunk_size=256)

big_chunks_retriever = ParentDocumentRetriever(
    vectorstore=vectorstore,
    docstore=store,
    child_splitter=child_splitter,
    parent_splitter=parent_splitter,
    search_kwargs={'filter': {'company': company}, 'k': NUM_PAR_CHUNKS}
)

In [44]:
from FlagEmbedding import FlagReranker

N_DOCS_RETURN = 2

reranker = FlagReranker('BAAI/bge-reranker-large', use_fp16=True) # Setting use_fp16 to True speeds up computation with a slight performance degradation

query = questions['question'][0]
passage = big_chunks_retriever.invoke(questions['question'][0])[0]
texts = []
for i in range(len(passage)):
  texts.append([query, passage[i]])

scores = reranker.compute_score(texts)
print(scores)

tokenizer_config.json:   0%|          | 0.00/443 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/279 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/801 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

OutOfMemoryError: CUDA out of memory. Tried to allocate 20.00 MiB. GPU 

In [ ]:
combined = list(zip(texts, scores))
sorted_combined = sorted(combined, key=lambda x: x[1], reverse=True)

top_n = 2
top_texts = [item[0] for item in sorted_combined[:top_n]]

In [ ]:
top_texts

In [ ]:
results_list = []

for i in tqdm(range(len(questions))):
  docs = top_texts
  generation = retrieval_generate.invoke({"context": docs, "question": questions['question'][i]})
  results_list.append(pd.DataFrame({
        'question': [questions['question'][i]],
        'response': [generation],
        'context': [docs]
    }))

results = pd.concat(results_list, ignore_index=True)